Change root directory

In [1]:
import os
os.chdir('/home/iaapp/brayan/helmholtzSim')
print(os.getcwd())

/home/iaapp/brayan/helmholtzSim


Import libraries

In [2]:
import time
import numpy as np
import pandas as pd
from src import helmCoils_simulator as sim
from src import plotMagneticField as hplot
from src import helmCoils_optmizer as opt


Initial square spire Simulation in X-Axis

In [3]:
number_of_spires = 2
size_length =  0.72
distance_among_spires = 0.46
turns = 30
current = 1 
rotation_matrix = np.eye(3)
X_coil = sim.CoilParameters(number_of_spires, size_length, distance_among_spires, turns, current, rotation_matrix)

# Simulation settings
parallel_coils = 150
batch_Size = 120
#grid_length_size = 0.01 #Space among evaluation points
grid_length_size = 0.01
num_seg = 100           #Numer of segments 

In [4]:
##Spawn spires
spires_x_sq = X_coil.square_spires(num_seg)

In [5]:
f0 = None
f0 = hplot.plot_spires(f0, spires_x_sq, color='black', row=None, col=None)

# Plot the square spires
f0.show()

In [6]:
# Generar grid
X, Y, Z = sim.generate_range([-(np.sum(X_coil.h)/2), 0], step_size_x = grid_length_size)

hplot.plot_grid(X, Y, Z, f0)

Optimization Problem

In [ ]:
# Make sure that Ax, X, Y, Z, and test_3 functions are properly defined in your environment.
optimizer = opt.HelmholtzOptimizer(
    desired_size = 0.30,          # 30e-2 is 0.30 m
    coil = X_coil,
    fun = X_coil.square_spires,
    #fun = X_coil.circular_spires,
    fix_L=False,
    fixed_L_value=2.0,          # Set a fixed value for L if needed
    grid_length_size=grid_length_size,
    population = 20,
    generations = 500,
    mutation = 0.6
)
optimizer.optimize()

gen	nevals	min 	avg   
0  	20    	0.07	500.11
1  	19    	0.07	0.106 
2  	17    	0.07	250.092
3  	19    	0.07	250.083
4  	14    	0.07	250.08 
5  	17    	0.07	250.076
6  	20    	0.07	250.075
7  	16    	0.07	0.093  
8  	13    	0.07	250.08 
9  	14    	0.07	250.079
10 	18    	0.07	0.08   
11 	18    	0.07	250.079
12 	13    	0.07	0.086  
13 	11    	0.07	0.084  
14 	16    	0.07	500.078
15 	13    	0.07	250.082
16 	13    	0.07	0.071  
17 	18    	0.07	0.0805 
18 	12    	0.07	500.077
19 	11    	0.07	0.0765 
20 	16    	0.07	0.083  
21 	16    	0.07	0.09   
22 	14    	0.07	0.08   
23 	13    	0.07	0.0865 
24 	12    	0.07	0.0805 
25 	11    	0.07	250.073
26 	16    	0.07	250.072
27 	14    	0.07	0.076  
28 	13    	0.07	0.073  
29 	16    	0.07	0.083  
30 	13    	0.07	500.069
31 	17    	0.07	250.082
32 	14    	0.07	0.078  
33 	15    	0.07	0.079  
34 	18    	0.07	250.078
35 	15    	0.07	0.0855 
36 	17    	0.07	0.08   
37 	19    	0.07	0.0795 
38 	16    	0.07	0.0815 
39 	19    	0.07	750.066
40 	16    	0.07	250